In [3]:
import pandas as pd
import numpy as np

In [2]:
"""On définit la fonction de nettoyage d'une vente"""

def nettoyage_vente(id, df):
    #id : indice de la vente
    #df : dataframe
    
    list_lots = ['lot1_surface_carrez', 'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez', 
                 'lot5_surface_carrez']
    
    mini_df = df.loc[(df['id_mutation']==id)] #on récupère la vente
    mini_df_appart = df.loc[(df['id_mutation']==id) & (df['appartement']==1)] #on récupère les appartements de cette vente
    index_app = mini_df_appart.index.tolist() #index des appartements
    
    nb_tot_app = mini_df_appart['appartement'].sum() #nombre total d'appartments dans la vente
    
    if nb_tot_app >= 1: #s'il y a au moins un appartement dans la vente, on essaie de récupérer pur chaque appart. la surface du lot associé
        
        mini_df_group_lot_surface = mini_df.groupby('lot1_numero').max()[list_lots] #pour chaque numéro de lot, on récupère la surface carrez max
        mini_df_group_lot_dep = mini_df.groupby('lot1_numero').sum()['dépendance'] #pour chaque numéro de lot, on récupère le nombre de dépendance

        for index in index_app:
            num_lot = mini_df.loc[index,'lot1_numero'] #on récupère le numéro de lot associé à l'appartement
            surface_carrez = mini_df_group_lot_surface.loc[num_lot].max() #on récupère la surface carrez associée au lot
            nb_dépendance = mini_df_group_lot_dep[num_lot] #on récupère le nombre de dépendances
            df.loc[index,'surface_carrez']=surface_carrez #on met à jour le dataframe
            df.loc[index,'nb_dépendances']=nb_dépendance #on met à jour le dataframe
            
        if nb_tot_app > 1: #s'il y a plus d'un appartement dans la vente, on met à jour le prix au prorata de la surface réelle
            surface_totale = mini_df_appart['surface_reelle_bati'].sum() #surface réelle totale
            prix_lot = mini_df_appart.loc[index_app[0]]['valeur_fonciere'] #on récupère la valeur foncière totale
            
            for index in index_app: #pour chaque appartement, on fait le prorata selon sa surface réelle
                surface_app = mini_df_appart.loc[index]['surface_reelle_bati'] #on récupère la surface de l'appartement
                df.loc[index, 'prix'] = prix_lot*(surface_app)/(surface_totale) #on met à jour le prix
                
        else: #si un seul appartement, rien à faire
            pass
            
    else: #si aucun appartement dans la vente, rien à faire
        pass

In [1]:
def nettoyage(emplacement_données):

    df_travail = pd.read_csv(emplacement, sep=",") #lecture du fichier

    variables_interet = ['id_mutation', 'date_mutation', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'ancien_id_parcelle', 'lot1_numero', 'lot1_surface_carrez',
       'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez',
       'lot5_surface_carrez', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales', 'longitude', 'latitude']

    list_lots = ['lot1_surface_carrez', 'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez', 
                 'lot5_surface_carrez']

    df_travail = df_travail.loc[:,variables_interet] #on ne retient que les variables qui nous intéressent
    df_travail = df_travail.loc[df_travail['nature_mutation']=='Vente'] #on ne garde que les ventes

    #On crée les variables supplémentaires dont on a besoin
    df_travail['appartement']=0.0
    df_travail['dépendance']=0.0
    df_travail['surface_carrez']=0.0
    df_travail['prix']=df_travail['valeur_fonciere'] #par défaut égal à la valeur foncière
    df_travail['nb_dépendances']=0.0
    df_travail['prix_au_m2_carrez']=0.0
    df_travail['prix_au_m2_réel']=0.0

    #on met à jour les variables binaires 'appartement' et 'dépendance'
    df_travail.loc[df_travail['type_local']=='Appartement', 'appartement']=1
    df_travail.loc[df_travail['type_local']=='Dépendance', 'dépendance']=1

    #par la suite on aura besoin de la variable 'lot1_numero' : on lui donne un format approprié, et on enlève les lignes qui ne la précisent pas
    #ces dernières sont peu nombreuses, et en général elles ne précisent pas non plus le type de bien immobilier, donc on peut les retirer
    df_travail['lot1_numero'] = pd.to_numeric(df_travail['lot1_numero'], errors='coerce')
    df_travail = df_travail.loc[df_travail['lot1_numero']>0]
    
    """On ne garde que les ventes d'appartement, 
    en prenant garde aux ventes groupées qui contiennent des appartments et des maisons et/ou des locaux industriels"""

    ventes_groupées = df_travail.groupby('id_mutation').count()['date_mutation'] #on rassemble selon l'indice
    ventes_groupées = ventes_groupées.loc[ventes_groupées>1].reset_index() #on trouve les ventes groupées

    id_ventes_à_supprimer = [] #liste qui va contenir les ventes à supprimer
    
    for id in ventes_groupées['id_mutation'].tolist():
        liste = df_travail.loc[df_travail['id_mutation']==id]['type_local'].tolist()
        if 'Maison' in liste:
            id_ventes_à_supprimer.append(id)
        elif 'Local industriel. commercial ou assimilé' in liste:
            id_ventes_à_supprimer.append(id)
        else:
            pass

    df_travail = df_travail.loc[~df_travail['id_mutation'].isin(id_ventes_à_supprimer)] #on supprime tous les indices identifiés
    df_travail = df_travail.loc[~df_travail['type_local'].isin(['Maison', 'Local industriel. commercial ou assimilé'])] #on supprime toutes les ventes isolées ne portant pas sur un appartement

    """Maintenant, on applique à chaque vente la fonction de nettoyage définie au-dessus et qui s'occupe de:
        - ajouter à chaque appartement le nombre de dépendances
        - ajouter la surface carrez si elle est disponible
        - faire le prorata du prix dans le cas où il y a plusieurs appartements"""

    for id in df_travail['id_mutation'].tolist():
        nettoyage_vente(id, df_travail)

    """Enfin, on fait quelques modifications"""

    var_fin = ['id_mutation', 'date_mutation', 'prix', 'type_local',
       'nb_dépendances', 'surface_reelle_bati', 'surface_carrez',
       'nombre_pieces_principales', 'prix_au_m2_carrez', 'prix_au_m2_réel', 'adresse_numero', 'adresse_nom_voie',
       'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune',
       'code_departement', 'id_parcelle', 'longitude', 'latitude']
    
    df_travail = df_travail.loc[df_travail['appartement']==1] #on ne garde que les appartements
    df_travail = df_travail.loc[:,var_fin] #on ne garde que les variables finales
    df_travail = df_travail[var_fin] #on réorganise l'ordre des variables
    df_travail['prix']=df_travail['prix'].round() #on arrondit les prix
    df_travail['prix_au_m2_carrez']= (df_travail['prix']/df_travail['surface_carrez']).round() #on met à jour le prix au m2 carrez
    df_travail['prix_au_m2_réel']= (df_travail['prix']/df_travail['surface_reelle_bati']).round() #on met à jour le prix au m2 réel
    df_travail = df_travail.rename(columns={'surface_reelle_bati':'surface_réelle', 'nombre_pieces_principales':'nb_pièces_principales'}) #on renomme la colonne surface_réelle_bati et le nombre de pièces

    #on ne garde que les lignes pour lesquelles on a un prix, une surface, une localisation
    df_travail = df_travail.loc[(df_travail['surface_réelle']>0) & (df_travail['longitude']>0) & (df_travail['prix']>0)]

    #on exporte les données en csv
    nom_fichier_sortie = f"{emplacement[:len(emplacement)-4]}_clean.csv"
    df_travail.to_csv(nom_fichier_sortie)

In [ ]:
#on nettoie les fichiers des années 2018 à 2021:

for année in ["2018","2019","2020","2021"]:
    emplacement = f"Projet-Stat-App-2A/Données/DVF_75_{année}.csv"
    nettoyage(emplacement)